An attempt of Adversarial Neuron Perturbation

Experiment with playing with model parameter tensors to achieve what we want

code references: \
https://www.kaggle.com/code/itsuki9180/introducing-adversarial-weight-perturbation-awp 

In [1]:
import torch
import torchvision

from torchsummary import summary

import numpy as np

In [2]:
torch.cuda.is_available()

True

In [3]:
#check gpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [58]:
import torchvision.transforms as transforms

cifar10_train = torchvision.datasets.CIFAR10('datasets/cifar_10', download=True, transform=transforms.ToTensor())
train_loader = torch.utils.data.DataLoader(cifar10_train, batch_size=10, shuffle=True, num_workers=4)

Files already downloaded and verified


In [148]:
res18 = torchvision.models.resnet18().cuda() if torch.cuda.is_available() else torchvision.models.resnet18()

In [149]:
# check if the parameters are in cuda
next(res18.parameters()).is_cuda

True

In [150]:
res18.load_state_dict(torch.load(f'saved_models/ResNet18-CIFAR10-Epoch-100.pth'))

C:\Users\Kevvol\AppData\Local\Temp\ipykernel_32072\2594117609.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  res18.load_state_dict(torch.load(f'saved_models/ResNet18-CI

<All keys matched successfully>

In [151]:
# print out named params
for name, param in res18.named_parameters():
    print(name)
    print(param.shape)

conv1.weight
torch.Size([64, 3, 7, 7])
bn1.weight
torch.Size([64])
bn1.bias
torch.Size([64])
layer1.0.conv1.weight
torch.Size([64, 64, 3, 3])
layer1.0.bn1.weight
torch.Size([64])
layer1.0.bn1.bias
torch.Size([64])
layer1.0.conv2.weight
torch.Size([64, 64, 3, 3])
layer1.0.bn2.weight
torch.Size([64])
layer1.0.bn2.bias
torch.Size([64])
layer1.1.conv1.weight
torch.Size([64, 64, 3, 3])
layer1.1.bn1.weight
torch.Size([64])
layer1.1.bn1.bias
torch.Size([64])
layer1.1.conv2.weight
torch.Size([64, 64, 3, 3])
layer1.1.bn2.weight
torch.Size([64])
layer1.1.bn2.bias
torch.Size([64])
layer2.0.conv1.weight
torch.Size([128, 64, 3, 3])
layer2.0.bn1.weight
torch.Size([128])
layer2.0.bn1.bias
torch.Size([128])
layer2.0.conv2.weight
torch.Size([128, 128, 3, 3])
layer2.0.bn2.weight
torch.Size([128])
layer2.0.bn2.bias
torch.Size([128])
layer2.0.downsample.0.weight
torch.Size([128, 64, 1, 1])
layer2.0.downsample.1.weight
torch.Size([128])
layer2.0.downsample.1.bias
torch.Size([128])
layer2.1.conv1.weight
tor

In [152]:
# all params should be either weights or biases
for name, param in res18.named_parameters():
    if not name.endswith('weight') and not name.endswith('bias'):
        # should never be called
        print(name)

In [153]:
# store named params and layers
params = {}
for name, param in res18.named_parameters():
    params[name] = param

layers = {}
for name, layer in res18.named_modules():
    layers[name] = layer

In [129]:
loss_fn = torch.nn.CrossEntropyLoss()

In [154]:
for name, layer in res18.named_modules():
    print(name)
    # print(layer.forward)


conv1
bn1
relu
maxpool
layer1
layer1.0
layer1.0.conv1
layer1.0.bn1
layer1.0.relu
layer1.0.conv2
layer1.0.bn2
layer1.1
layer1.1.conv1
layer1.1.bn1
layer1.1.relu
layer1.1.conv2
layer1.1.bn2
layer2
layer2.0
layer2.0.conv1
layer2.0.bn1
layer2.0.relu
layer2.0.conv2
layer2.0.bn2
layer2.0.downsample
layer2.0.downsample.0
layer2.0.downsample.1
layer2.1
layer2.1.conv1
layer2.1.bn1
layer2.1.relu
layer2.1.conv2
layer2.1.bn2
layer3
layer3.0
layer3.0.conv1
layer3.0.bn1
layer3.0.relu
layer3.0.conv2
layer3.0.bn2
layer3.0.downsample
layer3.0.downsample.0
layer3.0.downsample.1
layer3.1
layer3.1.conv1
layer3.1.bn1
layer3.1.relu
layer3.1.conv2
layer3.1.bn2
layer4
layer4.0
layer4.0.conv1
layer4.0.bn1
layer4.0.relu
layer4.0.conv2
layer4.0.bn2
layer4.0.downsample
layer4.0.downsample.0
layer4.0.downsample.1
layer4.1
layer4.1.conv1
layer4.1.bn1
layer4.1.relu
layer4.1.conv2
layer4.1.bn2
avgpool
fc


In [143]:
types = set()
for name, layer in res18.named_modules():
    types.add(type(layer))
    # print(layer.forward)

In [144]:
types

{torch.nn.modules.activation.ReLU,
 torch.nn.modules.batchnorm.BatchNorm2d,
 torch.nn.modules.container.Sequential,
 torch.nn.modules.conv.Conv2d,
 torch.nn.modules.linear.Linear,
 torch.nn.modules.pooling.AdaptiveAvgPool2d,
 torch.nn.modules.pooling.MaxPool2d,
 torchvision.models.resnet.BasicBlock,
 torchvision.models.resnet.ResNet}

In [132]:
fc_delta = torch.zeros_like(params['fc.weight'])
fc_xi = torch.zeros_like(params['fc.bias'])

fc_delta = fc_delta.uniform_(-0.05, 0.05)
fc_xi = fc_xi.uniform_(-0.05, 0.05)

In [133]:
fc_delta.requires_grad = True
fc_xi.requires_grad = True

In [157]:
conv1_delta = torch.zeros_like(params['conv1.weight'])
# conv1_xi = torch.zeros_like(params['conv1.bias'])

conv1_delta = conv1_delta.uniform_(-0.05, 0.05)
# conv1_xi = fc_xi.uniform_(-0.05, 0.05)

In [158]:
conv1_delta.requires_grad = True
# conv1_xi.requires_grad = True

In [135]:
from torch.nn import functional as F

def fc_forward_hook(module, i, o):
    # change the forward call
    return  F.linear(i[0], (1 + fc_delta) * module.weight, (1 + fc_xi) * module.bias)

In [159]:
def conv2d_forward_hook(module, i, o):
    # change the forward call
    return module._conv_forward(i[0], (1 + conv1_delta) * module.weight, module.bias)

In [160]:
# layer.register_forward_hook(fc_forward_hook)
layers['conv1'].register_forward_hook(conv2d_forward_hook)

In [161]:
for inputs, label in train_loader:
    inputs, label = inputs.to(device), label.to(device)
    
    pred = res18(inputs)
    loss = loss_fn(pred,label)
    loss.backward()
    # just 1 cycle and break
    break

In [162]:
# fc_delta.grad
conv1_delta.grad

tensor([[[[-0.0000e+00, -0.0000e+00, -0.0000e+00,  ..., -0.0000e+00,
           -0.0000e+00, -0.0000e+00],
          [-0.0000e+00, -0.0000e+00, -0.0000e+00,  ..., -0.0000e+00,
           -0.0000e+00, -0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
            0.0000e+00, -0.0000e+00],
          ...,
          [-0.0000e+00, -0.0000e+00, -0.0000e+00,  ..., -0.0000e+00,
           -0.0000e+00, -0.0000e+00],
          [-0.0000e+00, -0.0000e+00, -0.0000e+00,  ..., -0.0000e+00,
           -0.0000e+00, -0.0000e+00],
          [-0.0000e+00, -0.0000e+00, -0.0000e+00,  ..., -0.0000e+00,
           -0.0000e+00, -0.0000e+00]],

         [[-0.0000e+00, -0.0000e+00, -0.0000e+00,  ..., -0.0000e+00,
           -0.0000e+00, -0.0000e+00],
          [-0.0000e+00, -0.0000e+00, -0.0000e+00,  ..., -0.0000e+00,
           -0.0000e+00, -0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
            0.0000e+00, -0.0000e+00],
          ...,
     

In [163]:
# fc_delta.grad.shape
conv1_delta.grad.shape

torch.Size([64, 3, 7, 7])

In [165]:
conv1_delta.requires_grad = False

In [167]:
conv1_delta.grad = None

In [168]:
conv1_delta.grad

In [169]:
conv1_delta = conv1_delta.uniform_(-0.05, 0.05)

In [172]:
conv1_delta.clamp(-0.01, 0.01)

tensor([[[[-0.0100, -0.0100,  0.0100,  ..., -0.0100,  0.0023,  0.0100],
          [-0.0100,  0.0100, -0.0069,  ..., -0.0100, -0.0100,  0.0100],
          [-0.0077,  0.0100, -0.0100,  ..., -0.0100,  0.0020,  0.0100],
          ...,
          [ 0.0100,  0.0100,  0.0100,  ..., -0.0100, -0.0035,  0.0100],
          [-0.0100, -0.0100, -0.0100,  ...,  0.0100, -0.0100,  0.0100],
          [ 0.0100,  0.0100, -0.0100,  ..., -0.0100, -0.0092, -0.0048]],

         [[-0.0100,  0.0100, -0.0100,  ..., -0.0021, -0.0100,  0.0100],
          [ 0.0100, -0.0068,  0.0100,  ..., -0.0100,  0.0100, -0.0100],
          [ 0.0100,  0.0100,  0.0100,  ...,  0.0100,  0.0100, -0.0068],
          ...,
          [ 0.0100,  0.0100,  0.0100,  ...,  0.0100, -0.0022,  0.0100],
          [ 0.0100,  0.0100, -0.0100,  ...,  0.0100,  0.0100, -0.0029],
          [-0.0025,  0.0100, -0.0100,  ..., -0.0022,  0.0100,  0.0100]],

         [[-0.0064,  0.0100, -0.0100,  ...,  0.0100, -0.0089, -0.0100],
          [ 0.0041,  0.0096, -

In [173]:
torch.sign

<function torch._VariableFunctionsClass.sign>

In [175]:
conv1_delta.sign()

tensor([[[[-1., -1.,  1.,  ..., -1.,  1.,  1.],
          [-1.,  1., -1.,  ..., -1., -1.,  1.],
          [-1.,  1., -1.,  ..., -1.,  1.,  1.],
          ...,
          [ 1.,  1.,  1.,  ..., -1., -1.,  1.],
          [-1., -1., -1.,  ...,  1., -1.,  1.],
          [ 1.,  1., -1.,  ..., -1., -1., -1.]],

         [[-1.,  1., -1.,  ..., -1., -1.,  1.],
          [ 1., -1.,  1.,  ..., -1.,  1., -1.],
          [ 1.,  1.,  1.,  ...,  1.,  1., -1.],
          ...,
          [ 1.,  1.,  1.,  ...,  1., -1.,  1.],
          [ 1.,  1., -1.,  ...,  1.,  1., -1.],
          [-1.,  1., -1.,  ..., -1.,  1.,  1.]],

         [[-1.,  1., -1.,  ...,  1., -1., -1.],
          [ 1.,  1., -1.,  ...,  1.,  1., -1.],
          [-1.,  1., -1.,  ...,  1., -1.,  1.],
          ...,
          [-1., -1.,  1.,  ...,  1., -1., -1.],
          [-1., -1.,  1.,  ..., -1.,  1.,  1.],
          [ 1.,  1., -1.,  ..., -1., -1.,  1.]]],


        [[[-1., -1., -1.,  ...,  1., -1., -1.],
          [-1., -1.,  1.,  ...,  1.